In [183]:
from fuc import pyvcf
import pandas as pd 

In [187]:
txt_file = pd.read_csv('SP01_vep.txt', sep='\t', header=0)

In [190]:
txt_working = txt_file[['Location','Allele', 'Existing_variation']]
txt_working

,Location,Allele,Existing_variation
0,10:94426-94426,T,"rs10904045,COSV59118495"
1,10:94426-94426,T,"rs10904045,COSV59118495"
2,10:94426-94426,T,"rs10904045,COSV59118495"
3,10:94426-94426,T,"rs10904045,COSV59118495"
4,10:95429-95429,A,rs35442274
...,...,...,...
1084226,X:155238653-155238653,T,rs2472749
1084227,X:155238653-155238653,T,rs2472749
1084228,X:155238653-155238653,T,rs2472749
1084229,X:155238653-155238653,T,rs2472749


In [254]:
import re
def split_location(location):
    pattern = r'(\w+):(\d+)-(\d+)'
    match = re.match(pattern, location)
    if match:
        chromosome = match.group(1)
        position = match.group(2)
    else:
        return " "
            
    return chromosome, position


In [255]:
print(split_location('10:155238653-155238653'))

('10', '155238653')


In [264]:
print(split_location('X,111'))

In [265]:
pattern = r'(\w+):(\d+)-(\d+)'
search = re.search(pattern, 'X:155238653-155238653')
search.group(1)

'X'

In [258]:
def extract_rsID(variation):
    pattern = r'(\brs\d+)(?:,(\w+))?'
    match = re.match(pattern, variation)
    if match:
        rsID = match.group(1)       
        return rsID
    else:
        return " "

In [259]:
print(extract_rsID('rs10904045,COSV59118495'))

rs10904045


In [260]:
print(extract_rsID('rs10904045'))

rs10904045


In [261]:
print(extract_rsID('COSV59118495'))

In [198]:
assert extract_rsID('rs10904045,COSV59118495') == 'rs10904045'

In [199]:
txt_working = txt_working.assign(
        CHROM=txt_working['Location'].apply(lambda x: split_location(x)[0]),
        POS=txt_working['Location'].apply(lambda x: split_location(x)[1]),
        rsID=txt_working['Existing_variation'].apply(lambda x: extract_rsID(x)))

In [268]:
def get_header(vcf_filename):
    header_lines = []
    with open(vcf_filename, 'r') as file:
        for line in file:
            if line.startswith('#CHROM'):
                break
            header_lines.append(line.strip())
            
    header_line_str = '\n'.join(header_lines)
    header_line_str += '\n#'
    
    return header_line_str

In [269]:
print(get_header('SP01_vep.vcf'))

##fileformat=VCFv4.1
##VEP="v111" time="2024-02-13 09:59:17" cache="/net/isilonP/public/ro/ensweb-data/latest/tools/grch37/e111/vep/cache/homo_sapiens/111_GRCh37" db="homo_sapiens_core_111_37@hh-mysql-ens-grch37-web" 1000genomes="phase3" COSMIC="98" ClinVar="202306" HGMD-PUBLIC="20204" assembly="GRCh37.p13" dbSNP="156" gencode="GENCODE 19" genebuild="2011-04" gnomADe="r2.1" polyphen="2.2.2" regbuild="1.0" sift="sift5.2.2"
##INFO=<ID=CSQ,Number=.,Type=String,Description="Consequence annotations from Ensembl VEP. Format: Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|REF_ALLELE|UPLOADED_ALLELE|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID|MANE_SELECT|MANE_PLUS_CLINICAL|TSL|APPRIS|CCDS|SIFT|PolyPhen|AF|AFR_AF|AMR_AF|EAS_AF|EUR_AF|SAS_AF|gnomADe_AF|gnomADe_AFR_AF|gnomADe_AMR_AF|gnomADe_ASJ_AF|gnomADe_EAS_AF|gnomADe_FIN_AF|gnomADe_NFE_AF|gnomADe_OTH_AF|gnomADe_SAS_AF|g

In [214]:
def write_vcf(vcf_filename, update_df, new_file_path):
    header_line_str = get_header(vcf_filename)
    
    with open(new_file_path,'w') as file:
        file.write(header_line_str)
        
    update_df.to_csv(new_file_path, mode='a', sep='\t', index=False)

In [ ]:
from rsUpdate_vcf_regex import main

files_to_update = ['SP01', 'SP04', 'SP09', 'SP24']
for file in files_to_update:
    main(file)

/Users/chiranan/Desktop/Python/GitHub/VCF_Editing/repeat/rsUpdate_vcf_regex.py:40: DtypeWarning: Columns (69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  txt_file = pd.read_csv(f'{SPnumber}_vep.txt', sep='\t', header=0)


In [273]:
vcf_SP01_original = pyvcf.VcfFrame.from_file('SP01_vep.vcf')
vcf_SP01_original.df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,10,94426,.,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|TUBB8|ENSG000001...
1,10,95429,.,C,CA,. . .,.,CSQ=A|upstream_gene_variant|MODIFIER|TUBB8|ENS...
2,10,120756,.,A,T,. . .,.,CSQ=T|regulatory_region_variant|MODIFIER|||Reg...
3,10,126237,.,G,A,. . .,.,CSQ=A|downstream_gene_variant|MODIFIER|IL9RP2|...
4,10,126402,.,T,C,. . .,.,CSQ=C|non_coding_transcript_exon_variant|MODIF...
...,...,...,...,...,...,...,...,...
400592,X,155231738,.,T,TG,. . .,.,CSQ=G|intron_variant|MODIFIER|IL9R|ENSG0000012...
400593,X,155233098,.,T,C,. . .,.,CSQ=C|splice_polypyrimidine_tract_variant&intr...
400594,X,155234750,.,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|IL9R|ENSG0000012...
400595,X,155235014,.,C,T,. . .,.,CSQ=T|synonymous_variant|LOW|IL9R|ENSG00000124...


In [275]:
vcf_SP01_update = pyvcf.VcfFrame.from_file('SP01_vep_rsUpdate.vcf')
vcf_SP01_update.df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,10,94426,rs10904045,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|TUBB8|ENSG000001...
1,10,95429,.,C,CA,. . .,.,CSQ=A|upstream_gene_variant|MODIFIER|TUBB8|ENS...
2,10,120756,rs11253286,A,T,. . .,.,CSQ=T|regulatory_region_variant|MODIFIER|||Reg...
3,10,126237,rs2379071,G,A,. . .,.,CSQ=A|downstream_gene_variant|MODIFIER|IL9RP2|...
4,10,126402,rs1984905,T,C,. . .,.,CSQ=C|non_coding_transcript_exon_variant|MODIF...
...,...,...,...,...,...,...,...,...
400592,X,155231738,.,T,TG,. . .,.,CSQ=G|intron_variant|MODIFIER|IL9R|ENSG0000012...
400593,X,155233098,rs2037999,T,C,. . .,.,CSQ=C|splice_polypyrimidine_tract_variant&intr...
400594,X,155234750,rs191761523,C,T,. . .,.,CSQ=T|intron_variant|MODIFIER|IL9R|ENSG0000012...
400595,X,155235014,rs150608249,C,T,. . .,.,CSQ=T|synonymous_variant|LOW|IL9R|ENSG00000124...
